# Concepts Fondamentaux


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px # Pour installer le module Plotly, exécuter la commande ci-dessous

import random
import sys
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, RFECV, SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
IN_COLAB = 'google.colab' in sys.modules

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)

## Apprentissage supervisé
L'apprentissage supervisé est une technique d'apprentissage selon laquelle un modèle est entraîné sur un ensemble de données étiquetées. Cela signifie que chaque donnée d'entraînement est associée à une sortie correcte, connue sous le nom d'étiquette ou de cible. Le but de l'apprentissage supervisé est de permettre au modèle d'apprendre une fonction pouvant prédire les étiquettes de nouvelles données non étiquetées.

### A - Regression
L'objectif de la régression est de prédire une valeur continue à partir d'un ensemble de données.

L'entraînement du modèle permet l'apprentissage d'un ***Modèle*** prédit une valeur continue associée suivant d'une donnée. Un *bon* modèle est celui qui correspond au mieux à la distribution des données. <br>
Lors d'une regresion **linéaire** le modèle est représenté par une **droite** en 1D, un *plan* en 2D et un *hyperplan* en 3D ou plus. <br> 
Pour juger de la performance de notre modèle sur des données nouvelles, on va séparer nos données en un ensemble de test et un ensemble d'entraînement.  Pour ce faire, voici un exemple simple d'un modèle devant prédire la **température** en fonction de la **journée** dans l'année.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# Génération de données 
start_date = pd.Timestamp('2023-01-01')
end_date = pd.Timestamp('2023-12-31')
date_range = pd.date_range(start_date, end_date, freq='D')

def generate_temperature(date):
    day_of_year = date.timetuple().tm_yday
    seasonal = 10 * np.sin(2 * np.pi * day_of_year / 365.25)
    random_component = np.random.normal(0, 5)
    base_temperature = 15
    return base_temperature + seasonal + random_component

temperatures = [generate_temperature(date) for date in date_range]

data = pd.DataFrame({'Date': date_range, 'Temperature': temperatures})
data['NumericDate'] = (data['Date'] - start_date).dt.days  # Convertir de la date en valeur numérique

# Affichage de résultats
# Filtrer les données
start_month = 5  # Mai
end_month = 9  # Septembre
filtered_data = data[(data['Date'].dt.month >= start_month) & (data['Date'].dt.month <= end_month)]

plt.figure(figsize=(14, 6))
plt.scatter(filtered_data['Date'], filtered_data['Temperature'], alpha=0.5, label='Temperature')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.title('Scatter Plot des Températures')
plt.legend()
plt.show()

X = np.array(filtered_data['NumericDate']).reshape(-1, 1)  # Utiliser NumericDate au lieu de Date
y = filtered_data['Temperature']
reg = LinearRegression().fit(X, y)
y_pred = reg.predict(X)

# Tracer la droite de régression
plt.figure(figsize=(14, 6))
plt.scatter(filtered_data['Date'], filtered_data['Temperature'], alpha=0.5, label='Temperature')
plt.plot(filtered_data['Date'], y_pred, color='red', label='Regression Line')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.title('Régression Linéaire des Températures')
plt.legend()
plt.show()


Au travers de ce problème de regression *(Température vs les Jours de l'année)*,  on peut tracer la distribution conditionnelle **P(t|x, $ \mathbf{\vec{w}, \sigma^2} $ )** où $ \vec{\textit{w}} $ est le vecteur contennant les poids associés au modèle développé. 
<br> Afin d'illustrer **P(t|x, $ \mathbf{\vec{w}, \sigma^2} $ )**, on utilisera x = $\pm$ 13 Juillet 2023

In [ ]:
# Calcul de la distribution conditionnelle pour une date spécifique
target_date = pd.Timestamp('2023-07-13')
target_numeric_date = (target_date - start_date).days
target_prediction = reg.predict(np.array([[target_numeric_date]]))[0]

# Calcul de la variance résiduelle
residuals = y - y_pred
residual_variance = np.var(residuals)
residual_std = np.sqrt(residual_variance)

# Tracer la distribution conditionnelle
x_values = np.linspace(target_prediction - 3.25 * residual_std, 
                       target_prediction + 3.25 * residual_std, 100)
y_values = (1 / (np.sqrt(2 * np.pi * residual_variance))) * np.exp(-0.5 * ((x_values - target_prediction) ** 2 / residual_variance))

plt.figure(figsize=(14, 6))
plt.plot(x_values, y_values, label=f'P(t|x={target_date.date()}, w, $\sigma^2$)', color='blue')

# Lignes verticales pour la moyenne et les écarts types
plt.axvline(target_prediction, color='red', linestyle='--')
plt.axvline(target_prediction + residual_std, color='green', linestyle='--')
plt.axvline(target_prediction - residual_std, color='green', linestyle='--')
plt.axvline(target_prediction + 2 * residual_std, color='orange', linestyle='--')
plt.axvline(target_prediction - 2 * residual_std, color='orange', linestyle='--')
plt.axvline(target_prediction + 3 * residual_std, color='purple', linestyle='--')
plt.axvline(target_prediction - 3 * residual_std, color='purple', linestyle='--')

# Annotations des lignes
plt.annotate('Prediction (Mean)', xy=(target_prediction, 0), xytext=(target_prediction + 2, max(y_values) * 0.9),
             arrowprops=dict(facecolor='red', shrink=0.05), fontsize=10, color='red')
plt.annotate('Mean + 1 Std Dev', xy=(target_prediction + residual_std, 0), xytext=(target_prediction + residual_std + 2, max(y_values) * 0.8),
             arrowprops=dict(facecolor='green', shrink=0.05), fontsize=10, color='green')
plt.annotate('Mean - 2 Std Dev', xy=(target_prediction - 2 * residual_std, 0), xytext=(target_prediction - 2 * residual_std + 2, max(y_values) * 0.5),
             arrowprops=dict(facecolor='orange', shrink=0.05), fontsize=10, color='orange')
plt.annotate('Mean - 3 Std Dev', xy=(target_prediction - 3 * residual_std, 0), xytext=(target_prediction - 3 * residual_std + 2, max(y_values) * 0.3),
             arrowprops=dict(facecolor='purple', shrink=0.05), fontsize=10, color='purple')

plt.xlabel('Temperature')
plt.ylabel('Probability Density')
plt.title(f'Distribution Conditionnelle P(t|x={target_date.date()}, w, $\sigma^2$)')
plt.legend(loc='upper right')
plt.show()

Le modèle de Ridge vu en classe est tiré d'une régression par maximum a posteriori avec comme hypothèse que la distribution de vraisemblance **P(t|x, $ \mathbf{\vec{w}, \sigma^2} $ )** est **gaussienne**.  Voir les notes de cours pour plus de détails.

### Régression non linéaire
Il arrive souvent que le modèle linéaire est trop simple et n'arrive pas à correctement épouser les données. C'est le cas des données synthétiques ci-dessous.

In [ ]:
nb_data = 30
np.random.seed(1)
# Générer des données avec la fonction sinh
X_regP = np.linspace(-5, 5, nb_data)
X_regP = X_regP.reshape(-1, 1)
y_regP = np.sinh(X_regP)

noise_level = 6  # Ajuster le niveau de bruit
noise = np.random.normal(0, noise_level, size=X_regP.shape)
y_regP = y_regP + noise


# Séparation des données 
X_train, X_test, y_train, y_test = train_test_split(X_regP, y_regP, train_size=0.7, random_state=0)
modl= LinearRegression().fit(X_train, y_train)
print('Score du modèle:', modl.score(X_test, y_test))


# Affichage des données
plt.figure(figsize=(13, 5))
plt.subplot(1, 2, 1)
plt.scatter(X_test, y_test, color='blue', marker='o', label='Données de test')
plt.scatter(X_train, y_train, color='red', marker='o', label='Données d\'entraînement')
plt.plot([-8, 8], modl.predict(np.asarray([-8, 8])[:, None]), color='red')
plt.xlabel('x')
plt.ylabel('t')
plt.title('Régression linéaire')
plt.legend()
plt.show()


Comme on peut le remarquer avec la figure ci-dessus, notre modèle peine à réaliser de bonne performances. Pour ce faire, nous allons projeter nos données dans un espace plus grand que celui d'origine.  Nous ferons cela à l'aide d'une **fonction de base** ***polynomiale*** tel que vue en classe.  Nous entraînerons ensuite notre modèle sur ces nouvelles données.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge

alpha_regul = 0
degrees = np.array([1, 2, 4, 8, 10, 16, 19, 25])
X_plot = np.linspace(-5.1, 5.1, num=1000).reshape(-1, 1)
ymodl_plot = np.zeros((len(degrees), len(X_plot)))


for index, M in enumerate(degrees):
    modl = KernelRidge(alpha=alpha_regul, kernel='polynomial', degree=M).fit(X_train, y_train)
    ymodl_plot[index] = modl.predict(X_plot).flatten()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer les subplots
fig = make_subplots(rows=1, cols=2)
fig.update_yaxes(range = [-100,100])

# Ajouter des traces pour les points de test
fig.add_trace(
    go.Scatter(x=X_train.flatten(), y=y_train.flatten(), name="Points Train", mode='markers'), row=1, col=1
)
fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=y_test.flatten(), name="Points Test", mode='markers'), row=1, col=2
)

# Ajouter les modèles entraînés
for i, y_modl in enumerate(ymodl_plot):
    fig.add_trace(
        go.Scatter(x=X_plot.flatten(), y=y_modl.flatten(), name=f"Modèle Degree {2**i}(Train)", mode='lines', visible=False), row=1, col=1
    )
for i, y_modl in enumerate(ymodl_plot):
    fig.add_trace(
        go.Scatter(x=X_plot.flatten(), y=y_modl.flatten(), name=f"Modèle Degree {2**i}(Test)", mode='lines', visible=False), row=1, col=2
    )

# Créer les listes de visibilité pour chaque degré
num_models = len(ymodl_plot)
buttons = []
for i in range(num_models):
    visibility = [True, True] + [False] * (2 * num_models)
    visibility[2 + i] = True  # Montrer le modèle d'entraînement
    visibility[2 + num_models + i] = True  # Montrer le modèle de test
    buttons.append(
        dict(label=f"Degree {degrees[i]}",
             method="update",
             args=[{"visible": visibility},
                   {"title": "Comparaison des performances"}]
    ))

# Ajouter un bouton pour réinitialiser la visibilité
buttons.insert(0, dict(label="None",
                       method="update",
                       args=[{"visible": [True, True] + [False] * (2 * num_models)},
                             {"title": "None"}]))

# Mettre à jour la disposition avec le menu de sélection
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons
        )
    ],
    width=1200,
    height=600,
    title_text="Performance du modèle sur les données d'entraînement et de test"
)

# Afficher la figure
fig.show()


On peut constater que pour des valeurs élevées de **degree**, notre modèle à tendance à ***sur apprendre*** sur les données d'entraînement et par conséquent à mal performer sur l'ensemble de test. <br>
Afin de réduire l'effet du sur-apprentissage, on peut soit 1) augmenter le nombre de données d'entraînement et/ou 2) augmenter le terme de régularisation.  Pour ce faire, revenez en arrière et augmenter les valeurs de ***nb_data*** et de ***alpha_regul***

#### Régularisation
Notre prochain objectif est de déterminer  la valeur optimale de M à l'aide d'une validation croisée.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# Configurer k-fold cross-validation
scores = np.zeros((len(degrees), 2))
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
results_scores = np.ndarray(0)

# Tester chaque valeur de degrees
for M in degrees:
    model = KernelRidge(alpha=alpha_regul, kernel='polynomial', degree=M)
    scores = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    results_scores = np.append(results_scores,scores.mean())

# Afficher les résultats
for idx, M in enumerate(degrees):
    print(f"M: {M}, Mean Cross-Validation Score: {results_scores[idx]}")
print('\n\nLa degré M optimal du polynôme de la fonction de base est', degrees[np.argmax(results_scores)])
print('\n(À vérifier avec les graphiques précédents)')



### B - Classification Linéaire
La classification est une tâche qui consiste à prédire une catégorie ou une classe pour une observation donnée. Plus précisément, un modèle de classification apprend à partir de données étiquetées, c'est-à-dire un ensemble pour lequelle chaque observation est associé à une classe donnée.

Nous prendrons ici un ensemble de données très connu, à savoir **Iris**.  Le jeu de données Iris a été utilisé dans l'article classique de R.A. Fisher de 1936, *\"The Use of Multiple Measurements in Taxonomic Problems\"*.  Il inclut trois espèces d'iris (donc **trois classes** : *Setosa, Versicolor* et *Virginica*) avec 50 échantillons chacune ainsi que quatre propriétés pour chaque fleur : *longeur du sépale, largeur du sépale, longueur des pétales* et *largeur des pétales*. Une espèce de fleur est linéairement séparable des deux autres, mais les deux autres ne sont pas linéairement séparables l'une de l'autre.

"http://archive.ics.uci.edu/dataset/53/iris"

In [ ]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, chi2

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
y = iris.target

# Sélectionner les caractéristiques les plus import avec KBest
selector = SelectKBest(chi2, k=2)
X = selector.fit_transform(iris_df, y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Créer un scatter plot des données 
fig = px.scatter(
    x=X_scaled[:, 0], y=X_scaled[:, 1], color=iris.target,
    labels={'x': 'Feature 1', 'y': 'Feature 2'},
    width=800, height=600
)
fig.show()


In [ ]:
# Partitionnement des données 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

# Entraînement des données
svc = SVC(kernel='linear', random_state=42)
svc.fit(X_train, y_train)

# Préparation d'une grille de données
x_min, x_max = X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1
y_min, y_max = X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))
Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

print('Justesse moyenne du modèle: ', svc.score(X_test, y_test))

# Tracer les frontières de décision et les points de données
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.coolwarm)

# Tracer les points de données
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y, edgecolors='k', marker='o', cmap=plt.cm.coolwarm)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title("Résultat de la classification")
plt.show()



Il existe également des données ou la classification linéaire ne saurait être efficace. Dans ces cas de figure, comme vu au niveau de la régression, on projette les données dans un espace dimensionnelle plus grand dans lequel ces données pourrait faire objet d'une classification linéaire performante **ex:** *Méthode à noyaux, réseaux de neurones, ...*.

## Apprentissage non supervisée

L'apprentissage non supervisé porte sur les modèles entraînés sur des données non étiquetées, c'est-à-dire des données qui ne sont pas préalablement catégorisées. L'objectif dès lors est de découvrir la structure intrinsèque des données.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# Paramètres
n_samples = 500
n_features = 2
n_clusters = 4
noise_std = 1  # Écart type du bruit

# Génération des données
X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=n_clusters, random_state=42)
noise = np.random.normal(loc=0.0, scale=noise_std, size=X.shape)
X_noisy = X + noise

# Application du clustering KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
y_pred = kmeans.fit_predict(X_noisy)

# Affichage des clusters et leurs centroides
plt.figure(figsize=(10, 6))
plt.scatter(X_noisy[:, 0], X_noisy[:, 1], s=50, alpha=0.7)

plt.title('Clustering K-Means avec Bruit')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()


A vue d'oeil, on peut distinguer 4/5 ***clusters*** parmis nos données. On se sert par la suite de l'algorithme **Kmeans** qui permettra de regrouper les données de *même nature* autour de leur **centroïde**

In [ ]:
# Application du clustering KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
y_pred = kmeans.fit_predict(X_noisy)

# Affichage des clusters et leurs centroides
plt.figure(figsize=(10, 6))
plt.scatter(X_noisy[:, 0], X_noisy[:, 1], c=y_pred, cmap='viridis', s=50, alpha=0.7)

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.9, marker='X')

plt.title('Clustering K-Means avec Bruit')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()